In [1]:
import numpy as np
from typing import Callable
from numpy.typing import NDArray, ArrayLike
import csv
from datetime import datetime, timezone
import astropy
import pytz
from astropy.time import Time
import math
from pyquaternion import Quaternion
import pymap3d
import matplotlib.pyplot as plt
import ppigrf

In [2]:
def estimate_jacobian(f : Callable[NDArray[np.float64], NDArray[np.float64]],
                     state : NDArray[np.float64], #1 dimensional vector
                      epsilon = 1000 #limit definition of the derivative applied to vectorvalued partials
                     ):
    y = f(state)
    n = state.shape[0]#function input vector shape
    m = y.shape[0]#function output vector shape
    J = np.zeros((m, n))
    for i in range(n):
        state_1 = np.copy(state)
        state_1[i] = state[i] + epsilon
        f_1 = f(state_1)
        for j in range(m):
            J[j][i] = (f_1[j] - y[j])/epsilon

    return J

def raw_igdf_enu(height_from_sea_level : np.float64, lat : np.float64, lon : np.float64, time):
    #kilometers, degrees, degrees
    #returns B_east, b_north, b_up in nT: https://github.com/IAGA-VMOD/ppigrf/tree/main
    return ppigrf.igrf(lon, lat, height_from_sea_level, time)

def igdf_eci_vector(x : np.float64, y : np.float64, z : np.float64, time):
    #eci in METERS, returns unit eci igdf vector
    lat, lon, alt = pymap3d.eci2geodetic(x, y, z, time)
    b_east, b_north, b_up = raw_igdf_enu(alt/1000, lat, lon, time)
    ecefb_x, ecefb_y, ecefb_z = pymap3d.enu2ecef(b_east, b_north, b_up, lat, lon, alt)
    b_x, b_y, b_z = pymap3d.ecef2eci(ecefb_x, ecefb_y, ecefb_z, time)
    b_vec = np.array([b_x - x, b_y - y, b_z - z])
    return b_vec

In [111]:
dt = 10
def df(state):
    return np.array([
        state[3], state[4], state[5], state[6], state[7], state[8], 0, 0, 0
    ])
def f(state):
    return state + dt*df(state)

def h(state):
    return np.array([state[0], state[1], state[2]])

In [142]:
Q = np.eye(9)
R = np.eye(3) * 3

def EKF_iter(previous_state, previous_covariance, received_measurements):
    state_estimate = f(previous_state)
    F = estimate_jacobian(f, previous_state)
    covariance_estimate = F@previous_covariance@F.T + Q

    residual = received_measurements - h(state_estimate)
    H = estimate_jacobian(h, state_estimate)
    S = H@covariance_estimate@H.T + R
    K = covariance_estimate@H.T@np.linalg.inv(S)

    predicted_state = state_estimate + K@residual
    predicted_covariance = (np.eye(9) - K@H)@covariance_estimate
    return predicted_state, predicted_covariance
    

In [143]:
real_state = np.array([1000, 1000, 1000, 10, 0, 0, 0, 0, 0], dtype = np.float64)
measurements = h(real_state) + np.random.normal(size = 3)
noisy_state = np.array([1010, 970, 1020, 0, 0, 0, 0, 0, 0], dtype = np.float64)

estimated_J = estimate_jacobian(f, noisy_state, 1)

initial_covariance = np.linalg.inv(estimated_J.T@estimated_J)* 1000
for i in range(500):
    measurements = h(real_state) + np.random.normal(scale = 3, size = 3)
    noisy_state, initial_covariance = EKF_iter(noisy_state, initial_covariance, measurements)
    real_state = f(real_state)
print(real_state)
print(noisy_state)

[6000. 1000. 1000.   10.    0.    0.    0.    0.    0.]
[ 5.99065653e+03  1.00306770e+03  1.00165241e+03  1.23226021e+01
  1.05124866e+00  1.68745204e+00  1.15817913e+00 -7.38821974e-02
  5.20680502e-01]
